In [1]:
# ===============================
# Connect Four - Complete Simulation
# ===============================

import numpy as np
from itertools import combinations

# --- Board Functions ---
NUM_ROWS = 6
NUM_COLS = 7

def create_board():
    return np.zeros((NUM_ROWS, NUM_COLS), dtype=int)

def valid_moves(board):
    return np.where(board[0]==0)[0]

def make_move(board, col, player):
    for r in range(NUM_ROWS-1,-1,-1):
        if board[r,col]==0:
            board[r,col]=player
            return

def check_win(board, player):
    # horizontal
    for r in range(NUM_ROWS):
        for c in range(NUM_COLS-3):
            if np.all(board[r,c:c+4]==player):
                return True
    # vertical
    for c in range(NUM_COLS):
        for r in range(NUM_ROWS-3):
            if np.all(board[r:r+4,c]==player):
                return True
    # diagonal TL-BR
    for r in range(NUM_ROWS-3):
        for c in range(NUM_COLS-3):
            if np.all([board[r+i,c+i]==player for i in range(4)]):
                return True
    # diagonal BL-TR
    for r in range(3,NUM_ROWS):
        for c in range(NUM_COLS-3):
            if np.all([board[r-i,c+i]==player for i in range(4)]):
                return True
    return False

# --- Player Functions ---
def random_player(board, _):
    return np.random.choice(valid_moves(board))

def heuristic_player(board, player):
    opponent = 3 - player
    moves = valid_moves(board)
    # win if possible
    for move in moves:
        temp = board.copy()
        make_move(temp, move, player)
        if check_win(temp, player):
            return move
    # block opponent
    for move in moves:
        temp = board.copy()
        make_move(temp, move, opponent)
        if check_win(temp, opponent):
            return move
    # take center if possible
    center = NUM_COLS//2
    if center in moves:
        return center
    return np.random.choice(moves)

def intelligent_player(board, player, depth=1):
    opponent = 3 - player
    best_score = -np.inf
    best_move = valid_moves(board)[0]
    for move in valid_moves(board):
        temp = board.copy()
        make_move(temp, move, player)
        score = minimax(temp, depth-1, False, player, opponent)
        if score > best_score:
            best_score = score
            best_move = move
    return best_move

def minimax(board, depth, maximizing, player, opponent):
    if check_win(board, player): return 100
    if check_win(board, opponent): return -100
    if depth==0 or len(valid_moves(board))==0:
        return evaluate_board(board, player)
    if maximizing:
        return max(minimax(apply_move(board, move, player), depth-1, False, player, opponent)
                   for move in valid_moves(board))
    else:
        return min(minimax(apply_move(board, move, opponent), depth-1, True, player, opponent)
                   for move in valid_moves(board))

def apply_move(board, col, player):
    new_board = board.copy()
    make_move(new_board, col, player)
    return new_board

def evaluate_board(board, player):
    opponent = 3 - player
    score = 0
    for r in range(NUM_ROWS):
        for c in range(NUM_COLS-3):
            window = board[r,c:c+4]
            if np.count_nonzero(window==player)==3 and np.count_nonzero(window==0)==1:
                score += 10
            if np.count_nonzero(window==opponent)==3 and np.count_nonzero(window==0)==1:
                score -= 8
    return score

# --- Game Simulation ---
def simulate_game(player1_func, player2_func):
    board = create_board()
    turn = 1
    while True:
        move = player1_func(board, turn) if turn==1 else player2_func(board, turn)
        make_move(board, move, turn)
        if check_win(board, turn):
            return turn
        if len(valid_moves(board))==0:
            return 0
        turn = 3 - turn

# --- Monte Carlo with Pre-Sim ---
BASE_PRE_SIM = 50
MIN_SIM = 100
MAX_SIM = 500

players = {
    "4 Years": random_player,
    "7 Years": heuristic_player,
    "9 Years": lambda b,p: intelligent_player(b,p,depth=1),
    "11 Years": lambda b,p: intelligent_player(b,p,depth=2)
}

balance_estimates = {
    ("4 Years","7 Years"):0.1,
    ("4 Years","9 Years"):0.3,
    ("4 Years","11 Years"):0.05,
    ("7 Years","9 Years"):0.5,
    ("7 Years","11 Years"):0.7,
    ("9 Years","11 Years"):0.8
}

def determine_sim_count_from_pre_sim(pre_sim_outcomes):
    """Decide simulation count based on SE from pre-sim"""
    p1,p2,draw = pre_sim_outcomes
    se_max = max(np.sqrt(p1*(1-p1)/BASE_PRE_SIM),
                 np.sqrt(p2*(1-p2)/BASE_PRE_SIM),
                 np.sqrt(draw*(1-draw)/BASE_PRE_SIM))
    # map SE to simulation count linearly: high SE -> more sims
    sim_count = int(np.clip(100 + se_max*2000, MIN_SIM, MAX_SIM))
    return sim_count

# --- Run all pairings ---
for p1_name,p2_name in combinations(players.keys(),2):

    # --- Pre-Sim (50 games) ---
    pre_outcomes = {p1_name:0, p2_name:0, "Draw":0}
    for _ in range(BASE_PRE_SIM):
        winner = simulate_game(players[p1_name], players[p2_name])
        if winner==1: pre_outcomes[p1_name]+=1
        elif winner==2: pre_outcomes[p2_name]+=1
        else: pre_outcomes["Draw"]+=1
    pre_p1 = pre_outcomes[p1_name]/BASE_PRE_SIM
    pre_p2 = pre_outcomes[p2_name]/BASE_PRE_SIM
    pre_draw = pre_outcomes["Draw"]/BASE_PRE_SIM

    # --- Determine full simulation count ---
    sim_count = determine_sim_count_from_pre_sim((pre_p1, pre_p2, pre_draw))

    # --- Run full simulation ---
    outcomes = {p1_name:0, p2_name:0, "Draw":0}
    for _ in range(sim_count):
        winner = simulate_game(players[p1_name], players[p2_name])
        if winner==1: outcomes[p1_name]+=1
        elif winner==2: outcomes[p2_name]+=1
        else: outcomes["Draw"]+=1

    # --- Compute probabilities, SE, Quotes ---
    prob_p1 = outcomes[p1_name]/sim_count
    prob_p2 = outcomes[p2_name]/sim_count
    prob_draw = outcomes["Draw"]/sim_count
    se_p1 = np.sqrt(prob_p1*(1-prob_p1)/sim_count)
    se_p2 = np.sqrt(prob_p2*(1-prob_p2)/sim_count)
    se_draw = np.sqrt(prob_draw*(1-prob_draw)/sim_count)
    quote_p1 = 1/prob_p1 if prob_p1>0 else float('inf')
    quote_p2 = 1/prob_p2 if prob_p2>0 else float('inf')
    quote_draw = 1/prob_draw if prob_draw>0 else float('inf')

    # --- Print results ---
    print(f"\n{p1_name} vs {p2_name} ({sim_count} games)")
    print(f"{p1_name}: {prob_p1*100:.1f}% ± {se_p1*100:.1f}% → Quote ≈ {quote_p1:.2f}")
    print(f"{p2_name}: {prob_p2*100:.1f}% ± {se_p2*100:.1f}% → Quote ≈ {quote_p2:.2f}")
    print(f"Draw: {prob_draw*100:.1f}% ± {se_draw*100:.1f}% → Quote ≈ {quote_draw:.2f}")



4 Years vs 7 Years (100 games)
4 Years: 2.0% ± 1.4% → Quote ≈ 50.00
7 Years: 98.0% ± 1.4% → Quote ≈ 1.02
Draw: 0.0% ± 0.0% → Quote ≈ inf

4 Years vs 9 Years (198 games)
4 Years: 9.1% ± 2.0% → Quote ≈ 11.00
9 Years: 90.9% ± 2.0% → Quote ≈ 1.10
Draw: 0.0% ± 0.0% → Quote ≈ inf

4 Years vs 11 Years (155 games)
4 Years: 0.0% ± 0.0% → Quote ≈ inf
11 Years: 100.0% ± 0.0% → Quote ≈ 1.00
Draw: 0.0% ± 0.0% → Quote ≈ inf

7 Years vs 9 Years (100 games)
7 Years: 100.0% ± 0.0% → Quote ≈ 1.00
9 Years: 0.0% ± 0.0% → Quote ≈ inf
Draw: 0.0% ± 0.0% → Quote ≈ inf

7 Years vs 11 Years (240 games)
7 Years: 45.4% ± 3.2% → Quote ≈ 2.20
11 Years: 42.9% ± 3.2% → Quote ≈ 2.33
Draw: 11.7% ± 2.1% → Quote ≈ 8.57

9 Years vs 11 Years (100 games)
9 Years: 0.0% ± 0.0% → Quote ≈ inf
11 Years: 100.0% ± 0.0% → Quote ≈ 1.00
Draw: 0.0% ± 0.0% → Quote ≈ inf
